# 🧠 LibTorch Pipeline – Eğitimden Deploy'a Uçtan Uca Akış

Bu bölümde şu hattı baştan sona kuruyorum:

**Python (PyTorch) → Eğitim → Kaydet → TorchScript’e çevir → C++ (LibTorch + OpenCV) ile inference.**

Amaç:  
Gerçek bir CNN modeli eğitip, bu modeli C++ tarafında LibTorch kullanarak profesyonel şekilde çalıştırmak.

---

## 🔹 Genel Akış

1. **Proje yapısını hazırlıyorum.**
2. **Python tarafında eğitilebilir bir CNN modeli yazıyorum.**
3. CIFAR-10 dataset’i üzerinde **modeli eğitiyorum**.
4. En iyi ağırlıkları `best_model.pth` olarak kaydediyorum.
5. Aynı mimariyi kullanarak modeli **TorchScript formatına (`model_ts.pt`) export ediyorum.**
6. C++ tarafında:
   - LibTorch + OpenCV ile proje kuruyorum.
   - `model_ts.pt` dosyasını C++’ta yüklüyorum.
   - `cv::Mat → torch::Tensor` preprocess fonksiyonu yazıyorum.
   - TorchScript model üzerinde forward çalıştırıyorum.
   - Çıktıyı yorumlayıp (argmax) konsola veya görüntü üzerine yazıyorum.
   - İstersem kamera akışında canlı inference yapıyorum.

Bu akış; tipik bir “Python’da eğitim – C++’ta deploy” pipeline’ının sade bir prototipi.

---

## 🔹 Bölüm 1 – Python Tarafı: Model ve Dataset

Bu bölümde:

* CIFAR-10 dataset’ini torchvision ile indiriyorum.

* Görüntüleri 3×64×64 boyutuna getiriyorum (resize).

* Basit ama gerçekçi bir CNN modeli tanımlıyorum (3 → 32 → 64 → 128 kanallar, pooling, fc).

Yapı kabaca şöyle:

* Giriş: [B, 3, 64, 64]

* Conv blokları + MaxPool

* AdaptiveAvgPool → [B, 128, 4, 4]

* Flatten + Linear → [B, 10]

Bu mimari CIFAR-10 için fazlasıyla yeterli ve TorchScript ile uyumlu.

---

## 🔹 Bölüm 2 – Eğitim Döngüsü (Training Loop)

Eğitim tarafında yapacaklarım:

* Device seçimi: cuda varsa GPU, yoksa CPU.

* Loss: nn.CrossEntropyLoss

* Optimizer: Adam (learning rate ~ 1e-3)

EPOCHS kadar train loop:

* model.train() → forward, loss, backward, optimizer step.

* Epoch başına ortalama train loss ve accuracy hesaplıyorum.

* İstersem ayrıca val/test loop ekleyebilirim, ama temel amaç:
**makul bir doğruluk alan bir model çıkarmak.**

Eğitim sonunda:

* torch.save(model.state_dict(), "best_model.pth") ile ağırlıkları kaydediyorum.

* Bu dosyayı daha sonra TorchScript export için kullanacağım.

----
## Bölüm 3 – TorchScript Export (Python)

Eğitim bittikten sonra:

* Aynı CNN mimarisini yeniden kuruyorum.

* best_model.pth dosyasını load_state_dict ile yüklüyorum.

* Modeli eval() moduna alıyorum.

* torch.jit.trace ile örnek bir input üzerinden TorchScript oluşturuyorum.

* model_ts.pt olarak kaydediyorum.

Bu aşama sonrası:

* model_ts.pt, C++/LibTorch tarafından load edilebilir “donmuş” model haline geliyor.

* Python eğitim koduna gerek kalmadan C++ içinde sadece bu dosya ile inference yapabiliyorum.

---
## 🔹 Bölüm 4 – C++ Tarafı: LibTorch + OpenCV Projesi

C++ tarafında yapılacaklar:

* LibTorch CPU paketini kurdum ve C:/libtorch altına çıkardım.

OpenCV Windows build’ini kurdum; CMake için OpenCV_DIR olarak:

* C:/opencv/build/x64/vc16/lib yolunu kullanıyorum.

cpp_inference/CMakeLists.txt dosyasında:

* find_package(Torch REQUIRED)

* find_package(OpenCV REQUIRED)

* add_executable(model_test main.cpp)

* target_link_libraries(model_test "${TORCH_LIBRARIES}" ${OpenCV_LIBS})

* LibTorch ve OpenCV DLL’lerini POST_BUILD komutuyla exe’nin yanına kopyalıyorum.

Bu yapı sayesinde:

* cmake .. + cmake --build . ile model_test.exe’yi derleyebiliyorum.

* DLL sorunları yaşamadan LibTorch + OpenCV beraber çalışıyor.


---

# 🔹 Bölüm 5  – C++ Tarafı: Preprocess + Inference Mantığı

Bu bölümde, TorchScript formatına çevirdiğim modeli C++ içinde çalıştırmak için gerekli olan tüm adımları açıklıyorum. Amaç; görüntüyü tensöre dönüştürmek, modeli yüklemek, inference yapmak ve sonucun sınıfını hesaplamak.

---

### 📌 1) Modeli Yükleme

C++ tarafında TorchScript modelini yüklemek için:

```cpp
torch::jit::script::Module module = torch::jit::load("model_ts.pt");
module.to(torch::kCPU);
module.eval();
```
### Burada:

* load() → TorchScript dosyasını okur.

* to(torch::kCPU) → modeli CPU üzerinde çalıştırır.

* eval() → inference moduna alır (dropout, batchnorm stabilize olur).

### 📌 2) Görüntü Okuma (OpenCV)

C++ tarafında görüntüyü OpenCV ile yüklüyorum:

```cpp
cv::Mat img_bgr = cv::imread("test_image.jpg");
```

* OpenCV görüntüleri varsayılan olarak BGR formatında okur.
* TorchScript modeli ise RGB formatında çalıştığı için preprocess içinde renk dönüşümü yapacağım.

### 📌 3) Preprocess Adımları (BGR → Tensor)

* TorchScript modeli giriş olarak [1, 3, H, W] formatında bir tensör ister.
* Bu nedenle OpenCV’den gelen resmi PyTorch’un kabul edeceği forma dönüştürmek için şu adımları uyguluyorum:

* Resize → görüntüyü 64×64 boyutuna indiriyorum.

* BGR → RGB dönüşümü → cv::cvtColor kullanıyorum.

* uint8 → float32 → piksel değerlerini kayan noktalı tipe çeviriyorum.

* Normalize → değerleri 0–1 aralığına getiriyorum.

* HWC → CHW dönüşümü → OpenCV formatından PyTorch formatına taşıyorum.

* Batch dimension ekliyorum → [1, 3, H, W].

* clone() → Tensörün contiguous olmasını sağlıyorum, aksi halde hata oluşabilir.

Sonuç:
**Modelin girişine uygun, doğru formatta bir tensör elde etmiş oluyorum.**

### 📌 4) Inference Fonksiyonu

Model üzerinde forward çalıştırmak için:

>Tensörü std::vector<torch::jit::IValue> içine yerleştiriyorum.

* module.forward(inputs) fonksiyonunu çağırıyorum.

* Dönen değeri torch::Tensor olarak alıyorum.

* Bu işlem, Python tarafındaki:
```cpp
model(x)
```


* çağrısının C++ içindeki karşılığıdır.


### 📌 5) Postprocess (Çıktıyı Sınıfa Çevirme)

* Modelin döndürdüğü tensör genellikle şekil olarak [1, 10] veya [1, num_classes] şeklindedir.

Bu çıktıyı yorumlamak için:

Olasılık dağılımı:
```cpp
torch::softmax(output, 1);
```


Tahmin edilen sınıf index’i:
```cpp
output.argmax(1);
```


Bu sınıf index’i:

* Konsola yazılabilir,

* veya bir etiket listesiyle eşleştirilerek “cat”, “dog”, “car” gibi anlamlı sınıf isimlerine dönüştürülebilir.

**Bu işlemler tamamlandığında model, C++ içinde gerçek bir görüntü üzerinde çalışır hale gelir.**

--------
------------
------------
---------

----
----

--------
---------
-------
----


# 📌 Bölüm 1 – SimpleCNN Modeli ve CIFAR10 Dataset Hazırlığı

* Bu bölümde sınıflandırma için kullanacağım CNN mimarisini oluşturuyorum ve CIFAR10 dataset’ini yükleyip eğitim için hazırlıyorum.

Yapacaklarım:

1. SimpleCNN model mimarisini tanımlıyorum.
2. CIFAR10 dataset’ini indiriyorum.
3. Görüntüleri 64×64 boyutuna yeniden boyutlandırıyorum.
4. RGB → Tensor dönüşümü ve normalize işlemi yapıyorum.
5. DataLoader ile batch oluşturuyorum.
6. Modeli eğitime hazır hale getiriyorum.

Burada amaç:  
- C++ tarafında çalıştırabileceğim **TorchScript uyumlu**, sade ama iş yapan bir CNN modeli oluşturmak.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

# --------------------------
# Simple CNN Mimarisi
# --------------------------

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # [B, 32, 64, 64]
            nn.ReLU(),

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # [B, 64, 64, 64]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                          # [B, 64, 32, 32]

            nn.Conv2d(64, 128, kernel_size=3, padding=1),# [B, 128, 32, 32]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                          # [B, 128, 16, 16]
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                                # [B, 128*16*16]
            nn.Linear(128*16*16, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [4]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),           # 0-1 aralığı
    # Normalizasyon eklenecekse buraya eklenir
])

train_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("Dataset hazır, model oluşturuluyor...")

model = SimpleCNN(num_classes=10)
print(model)

100%|██████████| 170M/170M [02:35<00:00, 1.10MB/s] 


Dataset hazır, model oluşturuluyor...
SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=32768, out_features=256, bias=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=10, bias=True)
  )
)


----
 # 📌 Bölüm 2 – SimpleCNN Eğitim Döngüsü ve best_model.pth

Bu bölümde:

1. SimpleCNN modelimi CIFAR10 üzerinde eğitiyorum.
2. Eğitim süresince epoch bazında loss ve accuracy değerlerini izliyorum.
3. En iyi doğruluk (accuracy) elde edilen ağırlıkları `best_model.pth` dosyasına kaydediyorum.

Notlar:

- Eğitim cihazı: `cuda` varsa GPU, yoksa CPU.
- Loss fonksiyonu: `CrossEntropyLoss`
- Optimizasyon: `Adam`
- Amaç: Çok uzun süren, mükemmel bir eğitim değil;  
  **LibTorch deploy pipeline’ını besleyecek, makul bir model** elde etmek.


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

# Cihaz seçimi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanilan cihaz:", device)

# Modeli cihaza taşı
model = SimpleCNN(num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 1  # İstersen artırabilirsin (ör: 10)

best_acc = 0.0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = 100.0 * correct / total

    # Basit bir test/validation değerlendirmesi
    model.eval()
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = outputs.max(1)
            correct_test += preds.eq(labels).sum().item()
            total_test += labels.size(0)

    test_acc = 100.0 * correct_test / total_test

    print(f"Epoch [{epoch+1}/{EPOCHS}] | "
          f"Train Loss: {train_loss:.4f} | "
          f"Train Acc: {train_acc:.2f}% | "
          f"Test Acc: {test_acc:.2f}%")

    # En iyi modeli kaydet
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "best_model.pth")
        print(f"  -> Yeni en iyi model kaydedildi! (best_model.pth, acc={best_acc:.2f}%)")

print("Egitim tamamlandi. En iyi test accuracy:", best_acc)

Kullanilan cihaz: cpu
Epoch [1/1] | Train Loss: 1.4376 | Train Acc: 48.27% | Test Acc: 58.59%
  -> Yeni en iyi model kaydedildi! (best_model.pth, acc=58.59%)
Egitim tamamlandi. En iyi test accuracy: 58.59


---

Bu hücreyi çalıştırınca:

* Epoch epoch loss/accuracy göreceksin,

* En iyi test accuracy yakalandığında best_model.pth güncellenecek,

* Çalışma bittiğinde current directory’de best_model.pth olacak.

* Bu dosya, bizim TorchScript export için kaynak ağırlığımız.

----

# 📌 Bölüm 3 – TorchScript Export (best_model.pth → model_ts.pt)

Bu bölümde:

1. Aynı SimpleCNN mimarisini yeniden oluşturuyorum.
2. `best_model.pth` içindeki eğitilmiş ağırlıkları yüklüyorum.
3. Modeli `eval()` moduna alıyorum.
4. Örnek bir dummy input ile `torch.jit.trace` kullanarak TorchScript modeli üretiyorum.
5. Ortaya çıkan modeli `model_ts.pt` olarak kaydediyorum.

Bu `model_ts.pt` dosyası, C++ + LibTorch tarafında deploy için kullanacağım nihai model dosyasıdır.


In [10]:
import torch

# Aynı mimariyi yeniden kur
export_model = SimpleCNN(num_classes=10)
export_model.load_state_dict(torch.load("best_model.pth", map_location="cpu"))
export_model.eval()

# Örnek giriş (dummy input)
example_input = torch.randn(1, 3, 64, 64)

# TorchScript trace
traced_script_module = torch.jit.trace(export_model, example_input)

# Kaydet
traced_script_module.save("model_ts.pt")

print("TorchScript modeli kaydedildi: model_ts.pt")


TorchScript modeli kaydedildi: model_ts.pt


### Bu hücreyi çalıştırınca:

#### Dizinde iki önemli dosyan olacak:

* best_model.pth → eğitimde kullanılan ağırlıklar (sadece Python için)

* model_ts.pt → deploy/inference için C++’ta kullanacağımız TorchScript model

----

----
----

----

# 1) CMakeLists.txt (Uygulama - 5 kök dizininde)

Uygulama - 5 klasörünün içinde bir CMakeLists.txt oluştur ve içini tam aşağıdaki gibi yap:

```bash
cmake_minimum_required(VERSION 3.20 FATAL_ERROR)
project(cifar10_libtorch_inference)

# LibTorch kurulu yol
set(CMAKE_PREFIX_PATH "C:/libtorch")

# OpenCV kurulu yol
set(OpenCV_DIR "C:/opencv/build/x64/vc16/lib")

find_package(Torch REQUIRED)
find_package(OpenCV REQUIRED)

add_executable(model_test main.cpp)

# OpenCV include
include_directories(${OpenCV_INCLUDE_DIRS})

# Bağlanacak kütüphaneler
target_link_libraries(model_test
    "${TORCH_LIBRARIES}"
    ${OpenCV_LIBS}
)

set_property(TARGET model_test PROPERTY CXX_STANDARD 17)

# LibTorch ve OpenCV DLL'lerini exe yanına kopyala
add_custom_command(TARGET model_test POST_BUILD
    COMMAND ${CMAKE_COMMAND} -E copy_directory
        "C:/libtorch/bin"
        $<TARGET_FILE_DIR:model_test>
    COMMAND ${CMAKE_COMMAND} -E copy_directory
        "C:/opencv/build/x64/vc16/bin"
        $<TARGET_FILE_DIR:model_test>
)



# 2) main.cpp – Eğitimli model_ts.pt ile CIFAR10 Inference

Uygulama - 5 içinde bir main.cpp oluştur. İçine şunu koy:

```cpp
// ---------------------------------------------------------
// LibTorch + OpenCV ile TorchScript model inference örneği
// ---------------------------------------------------------
// Bu dosyada:
//  - TorchScript modeli (model_ts.pt) yüklüyorum
//  - OpenCV ile bir görüntü (test.jpg) okuyorum
//  - cv::Mat -> torch::Tensor (BGR -> RGB, resize, normalize) yapıyorum
//  - module.forward() ile tahmin alıyorum
//  - softmax + argmax ile sınıf index'ini hesaplıyorum
// ---------------------------------------------------------

#include <iostream>
#include <fstream>
#include <string>

#include <torch/script.h>      // TorchScript API
#include <opencv2/opencv.hpp>  // OpenCV ana header

// ---------------------------------------------------------
// cv::Mat (BGR) -> torch::Tensor [1, 3, H, W]
// ---------------------------------------------------------
// Adımlar:
//  1) Resize (H x W)
//  2) BGR -> RGB
//  3) uint8 -> float32, 0-1 normalize
//  4) HWC -> NHWC -> NCHW (PyTorch formatı)
//  5) clone() ile bellek güvence altına alma
torch::Tensor preprocess_image(const cv::Mat& img_bgr, int target_size) {
    cv::Mat img;

    // Orijinal görüntüyü kopyala (orijinale dokunmamak için)
    img_bgr.copyTo(img);

    // 1) Resize (W x H)
    cv::resize(img, img, cv::Size(target_size, target_size));

    // 2) BGR -> RGB (OpenCV BGR kullanır, model genelde RGB bekler)
    cv::cvtColor(img, img, cv::COLOR_BGR2RGB);

    // 3) uint8 -> float32 + normalize 0-1
    cv::Mat img_float;
    img.convertTo(img_float, CV_32F, 1.0f / 255.0f);

    // 4) HWC -> NHWC formatında tensöre çevir
    auto tensor_image = torch::from_blob(
        img_float.data,
        {1, img_float.rows, img_float.cols, 3},   // [N, H, W, C]
        torch::TensorOptions().dtype(torch::kFloat32)
    );

    // 5) NHWC -> NCHW (PyTorch standardı)
    tensor_image = tensor_image.permute({0, 3, 1, 2}); // [N, C, H, W]

    // contiguous + clone: OpenCV belleğinden ayrıştırmak için
    tensor_image = tensor_image.contiguous().clone();

    return tensor_image;
}

// ---------------------------------------------------------
// TorchScript module üzerinde inference fonksiyonu
// ---------------------------------------------------------
torch::Tensor run_inference(torch::jit::script::Module& module,
                            const torch::Tensor& input) {
    // TorchScript forward() IValue listesi bekler
    std::vector<torch::jit::IValue> inputs;
    inputs.push_back(input);

    // module.forward() -> IValue, toTensor() ile tensöre çevir
    torch::Tensor output = module.forward(inputs).toTensor();
    return output;
}

int main() {
    try {
        std::cout << "[INFO] Program basladi.\n";

        // ------------------------------
        // 1) Model dosya yolu
        // ------------------------------
        const std::string model_path = "model_ts.pt";

        // Dosya gerçekten var mı kontrol et
        {
            std::ifstream f(model_path);
            if (!f.good()) {
                std::cerr << "[HATA] model_ts.pt bulunamadi!\n";
                std::cerr << "       model_ts.pt dosyasini exe ile ayni klasore koy.\n";
                std::cout << "Enter'a basip cikabilirsin...\n";
                std::cin.get();
                return -1;
            }
        }

        // ------------------------------
        // 2) TorchScript modeli yükle
        // ------------------------------
        std::cout << "[INFO] Model yukleniyor...\n";
        torch::jit::script::Module module = torch::jit::load(model_path);

        // CPU kullanıyoruz (GPU varsa kCUDA da kullanılabilir)
        module.to(torch::kCPU);
        module.eval();
        std::cout << "[OK] Model yuklendi ve eval modunda.\n";

        // ------------------------------
        // 3) Test resmi oku
        // ------------------------------
        const std::string image_path = "test.jpg";  // exe ile aynı klasörde olmalı
        cv::Mat img_bgr = cv::imread(image_path);

        if (img_bgr.empty()) {
            std::cerr << "[HATA] Resim yuklenemedi: " << image_path << "\n";
            std::cerr << "       test.jpg dosyasini exe ile ayni klasore koy.\n";
            std::cout << "Enter'a basip cikabilirsin...\n";
            std::cin.get();
            return -1;
        }

        std::cout << "[OK] Resim yuklendi. Boyut: "
                  << img_bgr.cols << "x" << img_bgr.rows << " (W x H)\n";

        // ------------------------------
        // 4) Preprocess -> Tensor
        // ------------------------------
        int target_size = 64; // Python tarafında Resize((64,64)) kullandık
        torch::Tensor input_tensor = preprocess_image(img_bgr, target_size);

        std::cout << "[DEBUG] Input tensor shape: " << input_tensor.sizes() << "\n";

        // Beklenen shape: [1, 3, 64, 64]
        auto sizes = input_tensor.sizes();
        if (!(sizes.size() == 4 &&
              sizes[0] == 1 &&
              sizes[1] == 3 &&
              sizes[2] == target_size &&
              sizes[3] == target_size)) {
            std::cerr << "[HATA] Input tensor beklenenden farkli! Gelen: "
                      << sizes << "\n";
            std::cout << "Enter'a basip cikabilirsin...\n";
            std::cin.get();
            return -1;
        }

        // ------------------------------
        // 5) Inference (forward)
        // ------------------------------
        std::cout << "[INFO] Inference calistiriliyor...\n";
        torch::Tensor output = run_inference(module, input_tensor);

        std::cout << "[DEBUG] Output tensor shape: " << output.sizes() << "\n";

        // ------------------------------
        // 6) Postprocess: softmax + argmax
        // ------------------------------
        // Çıkış tensörü: [1, num_classes] (örn. [1, 10])
        torch::Tensor probs = torch::softmax(output, 1);   // sınıf olasılıkları
        torch::Tensor pred_class = probs.argmax(1);        // en yüksek olasılıklı sınıf index'i

        std::cout << "Output (logits): " << output << "\n";
        std::cout << "Predicted class index: "
                  << pred_class.item<int>() << "\n";

        std::cout << "[INFO] Program bitti. Enter'a basip cikabilirsin...\n";
        std::cin.get();
    }
    catch (const c10::Error& e) {
        std::cerr << "[EXCEPTION - c10] " << e.what() << "\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }
    catch (const std::exception& e) {
        std::cerr << "[EXCEPTION - std] " << e.what() << "\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }

    return 0;
}


---
# 3) Dosya Yerleşimi (Önemli)

Şu anda:
```bash
Uygulama - 5/
  CMakeLists.txt
  main.cpp
  model_ts.pt        (Python'dan buraya kopyala)
  test_image.jpg     (herhangi bir CIFAR-like resim koy)
  build/             (birazdan oluşacak)
```


İlk derlemeden sonra:
```bash
Uygulama - 5/
  ├─ CMakeLists.txt
  ├─ main.cpp
  ├─ model_ts.pt
  ├─ test_image.jpg
  └─ build/
       └─ Debug/
            ├─ model_test.exe
            ├─ model_ts.pt      (sonradan buraya kopyalayacaksın)
            ├─ test_image.jpg   (sonradan buraya kopyalayacaksın)
            └─ (DLL'ler)
```


Çalıştırılan exe, build/Debug içinde olduğu için:

* model_ts.pt’yi

* test_image.jpg’yi

* build/Debug içine de kopyalaman lazım.

Yoksa program model dosyasını/resmi bulamaz.

---

# 4) Derleme ve Çalıştırma Komutları

Terminali aç:
```bash
cd "C:\Users\hdgn5\OneDrive\Masaüstü\C ++\Bölüm - 11 ( LibTorch )\Uygulama - 5"

mkdir build
cd build
cmake ..
cmake --build .
```


Sonra:
```bash
cd Debug
model_test.exe
```


Beklediğimiz konsol çıktısı türü:

* [INFO] Program basladi.

* [OK] Model yüklendi ve CPU uzerinde eval moduna alindi.

* [OK] Orijinal resim boyutu: ...

* [DEBUG] Input tensor shape: [1, 3, 64, 64]

* [DEBUG] Output tensor shape: [1, 10]

* Predicted class index: X
---